In [1]:
import pyautogui as at
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import pandas as pd
import requests
import bs4

In [2]:
class webdriverGet():
    extension_path1 = './LogPrograme/3.0.8_0.crx'
    extension_path2 = './LogPrograme/13.0.10_0.crx'
    download_path = "D:\VScode\github\peenutty-1688\filses_download"
    def __init__(self):
        try:
            self.web_options = webdriver.ChromeOptions()
            self.web_options.add_extension(self.extension_path1)
            self.web_options.add_extension(self.extension_path2)
            self.web_options.add_experimental_option("prefs", {
                "download.default_directory": self.download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            })
        except:
            return {"message":"error set Webdriver","status":404}
    
    def login_1688(self):
        def click_google():
            while(True):
                try:
                    self.driver.switch_to.window(self.driver.window_handles[0])
                    self.driver.find_element(By.XPATH,f"/html/body/div[1]/div/div/div[2]/section/div[2]/a[3]/figure").click()
                    return 200
                except: pass
        def write_email():
            while(True):
                try:
                    self.driver.find_element(By.XPATH,f'//*[@id="identifierId"]').send_keys(self.my_email+Keys.ENTER)
                    return 200
                except: pass
        def write_password():
            while(True):
                try:
                    self.driver.find_element(By.XPATH,f'//*[@id="password"]/div[1]/div/div[1]/input').send_keys(self.my_password+Keys.ENTER)
                    return 200
                except:pass
        click_google()
        time.sleep(3)
        write_email()
        time.sleep(3)
        write_password()
        return {"status": 200}

    def open(self):
        try:
            self.driver = webdriver.Chrome(options=self.web_options)
        except:
            return {"message":"error open webdriver","status":404}
    
    def closeTab(self):
        tab = self.driver.window_handles
        count_tab = len(tab)
        if(count_tab>1):
            for i in range(count_tab-1,0,-1):
                self.driver.switch_to.window(self.driver.window_handles[i])
                self.driver.execute_script("window.close();")

    def ToLink(self,url):
        try:
            self.driver.switch_to.window(self.driver.window_handles[0])
            self.driver.get(url=url)
        except:
            return {"message":"error get Url","status":404}
        
    def close(self):
        try:
            self.driver.quit()
        except:
            return {"message":"error close webdriver","status":404}

    def checkZero(self,_time):
        try:
            # select 0 
            self.driver.switch_to.window(self.driver.window_handles[-1])
            self.data = self.driver.page_source
            self.soup = bs4.BeautifulSoup(self.data)
            for i in range(20,0,-1):
                time.sleep(_time)
                self.get_number = self.soup.select_one(f'body > div > div > div > div.ap-page__bd > div > div.ap-compare-products__bd > div > table > tbody:nth-child(3) > tr:nth-child(3) > td:nth-child({i+1}) > div')
                try:
                    if int(self.get_number.text) == 0:
                        self.driver.find_element(By.XPATH,f"/html/body/div/div/div/div[2]/div/div[2]/div/table/thead/tr/th[{i+1}]/div[3]").click()
                except:
                    return {"message":"ไม่พบสินค้า...","status":200}
            return {"message":"เช็ค 0 สินค้าสำเร็จ","status":200}
        except:
            return {"message":"ไม่พบหน้าที่ต้องการให้เช็ค 0","status":404}
        
    def select_item20(self,_time):
        try:
            # set web index 0
            self.driver.switch_to.window(self.driver.window_handles[0])
            # click 20 item
            for index in range(20):
                time.sleep(_time)
                self.driver.find_element(By.XPATH,f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+1}]/div/div[10]').click()
                self.element = self.driver.find_element(By.XPATH, f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+2}]')
                self.driver.execute_script("arguments[0].scrollIntoView();", self.element)
            time.sleep(_time)
        except:
            return {"message":"error select item","status":404}

    def DownloadExcel(self):
        try:
            # 
            # download excel
            self.driver.find_element(By.XPATH,'/html/body/div/div/div/div[2]/div/div[1]/div/div').click()
        except:
            return {"message":"error Download file","status":404}
    
    def saveUrl(self,name):
        try:
            self.driver.switch_to.window(self.driver.window_handles[-1])
            open(f'{name}.txt','w',encoding='utf-8').write(self.driver.current_url)
        except:pass

<>:4: SyntaxWarning: invalid escape sequence '\V'
<>:4: SyntaxWarning: invalid escape sequence '\V'
C:\Users\Passa\AppData\Local\Temp\ipykernel_14640\337097815.py:4: SyntaxWarning: invalid escape sequence '\V'
  download_path = "D:\VScode\github\peenutty-1688\filses_download"


หลังจากที่เปิดเว็บให้ทำการล็อคอิน 1688 ก่อนถึงจะสามารถใช้งานต่อได้

In [41]:
import tkinter as tk
from tkinter import filedialog

def select_path():
  path = filedialog.askdirectory()
  if path:
    label["text"] = f"{path}"
def showPath():
  print(label["text"])
  
# สร้างหน้าต่าง tkinter
root = tk.Tk()
root.title("เลือก Path")

# สร้างปุ่มสำหรับเลือก path
button = tk.Button(text="เลือก Path", command=select_path)
button.pack()

# สร้าง label สำหรับแสดง path
label = tk.Label(text="")
label.pack()

button2 = tk.Button(text="show Path", command=showPath)
button2.pack()
# รันหน้าต่าง tkinter
root.mainloop()


D:/VScode/github/peenutty-1688/filses_download


In [2]:
extension_path1 = './LogPrograme/3.0.8_0.crx'
extension_path2 = './LogPrograme/13.0.10_0.crx'
download_path = "D:/VScode/github/peenutty-1688/filses_download"
download_path = download_path.replace("/","\\")
web_options = webdriver.ChromeOptions()
web_options.add_extension(extension_path1)
web_options.add_extension(extension_path2)
web_options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    # "download.prompt_for_download": False,
    # "download.directory_upgrade": True,
    # "safebrowsing.enabled": True,
    
})
driver = webdriver.Chrome(options=web_options)
driver.get('https://www.aliprice.com/Member/login.html?ext_id=10100&platform=1688&version=3.0.8&browser=chrome&channel=chrome&mv=3&redirect=https%3A%2F%2Fwww.1688.com')
# driver.window_handles
driver.switch_to.window(driver.window_handles[-1])
driver.execute_script("window.close();")
time.sleep(1)
driver.switch_to.window(driver.window_handles[-1])
driver.execute_script("window.close();")
driver.switch_to.window(driver.window_handles[0])


In [5]:
# Login 1688
# email : tmhumr1frz2vjv7nf1i62lc@sharklasers.com
# password : passapol47

In [21]:
driver.get('chrome-extension://lenipkahddombkldhfmcnnjakmcepdlk/options.html')

In [3]:
url = 'http://localhost:3030/'
img = 'https://down-th.img.susercontent.com/file/th-11134207-7r98z-lo1hlrzwom7b75'
driver.get(f'{url}?url={img}')

In [4]:
position_img = (450,494)
downcount = 7
at.click(position_img)
at.rightClick()
time.sleep(1)
for i in range(downcount):
    at.press('down')
at.press('enter')
at.press('enter')

In [5]:
for index in range(20):
    # try:
    #     driver.find_element(By.XPATH,f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{1}]/div/div[10]').click()
    # except:
    #     driver.find_element(By.XPATH,f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{1}]/div/div[9]').click()
    
    # driver.find_element(By.XPATH,f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+1}]/div/div[10]').click()
    driver.find_element(By.XPATH,f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+1}]/div/div[9]').click()
    element = driver.find_element(By.XPATH, f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+2}]')
    driver.execute_script("arguments[0].scrollIntoView();", element)
driver.find_element(By.XPATH,f'/html/body/div[2]/div/div[1]/div[2]/div/div[2]').click()

In [10]:
time.sleep(30);

In [6]:
# at.position()
positionDowload = (1118,297) 
at.click(positionDowload)

In [8]:
driver.switch_to.window(driver.window_handles[-1]);
driver.current_url
driver.execute_script("window.close();")
driver.switch_to.window(driver.window_handles[0]);

chrome-extension://lenipkahddombkldhfmcnnjakmcepdlk/compareProducts.html?data=...%3D%3D

In [ ]:
import os,csv,pandas as pd,numpy as np

def readCsvAll(folder_path='./'):
    csv_files = []
    try:
        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                csv_files.append(filename)
    except:pass
    return csv_files
def readFileCsv(name):
    dataCsv = []
    try:
        with open(name, mode='r', newline='', encoding='utf-8') as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                dataCsv.append(row)
    except:pass
    return dataCsv
#////////////////////////////////////////////////////////////////
try:
    df = pd.read_excel('./dataHead.xlsx')
    header = df.columns
    DataFileExcel = [] 
    for idx in range(len(df.values)):
        values = df.values[idx]
        DataFileExcel.append(values)
#////////////////////////////////////////////////////////////////
    DataResult = []
    for dataRead in DataFileExcel:
        dataStart = dataRead 
        for r in range(3):
            dataStart = np.append(dataStart,'')
        id = dataStart[0]
        s = readFileCsv(f'{id}.csv')
        if s:
            for idx in range(23):
                if idx == 0:
                    dataStart = np.append(dataStart,s[1::][idx])
                    DataResult.append(dataStart)
                else:
                    list1 = ['' for i in range(12)]
                    if(idx < len(s[1::])):
                        list1 = np.append(list1, s[1::][idx])
                    DataResult.append(list1)
#////////////////////////////////////////////////////////////////
    dataTest = pd.DataFrame(DataResult)
    dataTest.to_excel('./ResultData.xlsx')
except:
    open('./dataHead.xlsx', 'w',encoding='utf8')